<a href="https://colab.research.google.com/github/Mohamed-ASHO-Hassan/WideBot-getting-to-philosophy/blob/master/getting_to_philosophy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import requests, re
from bs4 import BeautifulSoup
import time

In [ ]:

def find_philosophy(url):
  url_o="_"
  MAX_HOPS = 100 
  count = 0 # number of hops

  r = requests.get(url)
   
  soup = BeautifulSoup(r.text)
  print(r.url) # Print current url (after redirection)
  s1=soup.find('h1',{'id':'firstHeading'})

  
  while s1.text != 'Philosophy':
  #for i in range( 1) :
    if count==MAX_HOPS:
      print("MAX_HOPS reached.")
      return None

    content = soup.find(id='mw-content-text')

    
    for t in content.find_all(class_=['navbox', 'vertical-navbox', 'toc']):
      t.replace_with("")
    
    paragraph = soup.find_all()[0] # Only DIRECT child
    paragraph.findChildren( recursive=False)

    
    for s in paragraph.find_all(['span', 'small', 'sup,', 'i', 'table']): # remove spans and smalls with language, pronounciation
      s.replace_with("")
    paragraphText = str(paragraph)
    paragraphText = re.sub(r' \(.*?\)', '', paragraphText) # Remove leftover parenthesized text 

    reParagraph = BeautifulSoup(paragraphText) # back into bs4 object to find links
    firstLink = reParagraph.find_all(href = re.compile('^/wiki/'))[0] # links that start with /wiki/ only
    

    
    while firstLink == None :
      # case of disambiguation: use first wiki link in list
      if '(disambiguation)' in url or '(surname)' in url or '(Protection_policy#semi)' in url:
        firstLink = content.ul.find(href = re.compile('^/wiki/'))

      else:  
        paragraph = paragraph.find_next_sibling("p")
        
        if(paragraph is None): # Catch-case

          if(content.ul is not None):
            firstLink = content.ul.find(href = re.compile('^/wiki/')) # Disambiguation-type page
          if(firstLink is None): # No links available
            print("Wikipedia not reachable.")
            return None
          continue

        for s in paragraph.find_all(['span', 'small', 'sup,', 'i', 'table']):
          s.replace_with("")
        paragraphText = str(paragraph)
        paragraphText = re.sub(r' \(.*?\)', '', paragraphText)
        reParagraph = BeautifulSoup(paragraphText)
        firstLink = reParagraph.find(href = re.compile('^/wiki/'))

      # For debugging:
      #print(paragraphText) 

    url = 'http://en.wikipedia.org' + firstLink.get('href')

    ##################################################################################case 1
    if url=="http://en.wikipedia.org/wiki/Wikipedia:Protection_policy#semi":     
      print("not reachable.")
      return None

   ##################################################################################case 2
    if (count%2==0):
      if url_o==url:     
        print("Entered a loop")
        return None
      else:        
        url_o=url

    print(url)
    r = requests.get(url) # Make new request
    soup = BeautifulSoup(r.text) # Soup it up again

    time.sleep(0.5)
    count = count+1

  print(str(count) + " hops")
  return count
  

url='https://en.wikipedia.org/wiki/Special:Random'

find_philosophy(url)

https://en.wikipedia.org/wiki/John_Q._Farmer
http://en.wikipedia.org/wiki/Minnesota
http://en.wikipedia.org/wiki/Wikipedia:Featured_articles
not reachable.
